In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 160.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [3]:
from datasets import Dataset
import pandas as pd
df = pd.read_csv("merged_dataset_new.csv")

In [4]:
df['input'] = 'Type: '+ df['Type'] + ",  Statement: "+ df['Statement'] + ",  Reference data: " + df['Reference_data']
df['output'] = "Label: "+ df['Label'] + ",  Primary Evidence: "+ df['Primary_evidence_index'] + ",  Secondary Evidence: " + df['Secondary_evidence_index']
df = df[['input', 'output']]
df['output'] = df['output'].fillna("Label: None")
dataset = Dataset.from_pandas(df)

<ipython-input-4-ffe4bedf55c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['output'] = df['output'].fillna("Label: None")


In [26]:
# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float32"  
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training Arguments
output_dir = "./results"
per_device_train_batch_size = 1 
gradient_accumulation_steps = 16  
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 10
logging_steps = 25
max_seq_length = 512  
packing = False
device_map = {"": 0}

compute_dtype = torch.float32  

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_name = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 

In [27]:

def tokenize_function(examples):
    inputs = ["<s> [INST]<<SYS>>\nSystem prompt\n<</SYS>>\n" + inp + " [/INST] " for inp in examples['input']]
    outputs = [out + " </s>" for out in examples['output']]
    return tokenizer(inputs, text_target=outputs, padding="max_length", truncation=True, max_length=max_seq_length)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/1700 [00:00<?, ? examples/s]

In [30]:

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,  
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,   
    optim=optim,
    save_steps=10_000,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False, 
    bf16=False, 
    max_grad_norm=1.0,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=True, 
)


In [31]:
for param in model.parameters():
    param.data = param.data.to(torch.float32)
    if param.grad is not None:
        param.grad.data = param.grad.data.to(torch.float32)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


Step,Training Loss
25,0.000000
50,0.000000
75,0.000000
100,0.000000


TrainOutput(global_step=106, training_loss=0.0, metrics={'train_runtime': 2510.0778, 'train_samples_per_second': 0.677, 'train_steps_per_second': 0.042, 'total_flos': 1.7554643601063936e+16, 'train_loss': 0.0, 'epoch': 1.0})

In [2]:
import pandas as pd
df_predict = pd.read_csv('Gold_test_dataset.csv')
df_predict['combined'] = "Type: " + df_predict['Type'] + " Statement: " + df_predict['Statement'] + " Reference_data: " + df_predict['Reference_data']
prompt = df_predict['combined'][1]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

fine_tune_Llama_2 = "./fine_tune_Llama_2"

model = AutoModelForCausalLM.from_pretrained(fine_tune_Llama_2, ignore_mismatched_sizes=True)

tokenizer = AutoTokenizer.from_pretrained(fine_tune_Llama_2)

print("Model and tokenizer loaded successfully.")


In [ ]:
formatted_input = "<s> [INST]<<SYS>>\nSystem prompt\n<</SYS>>\n" + prompt + " [/INST] "

inputs = tokenizer(formatted_input, return_tensors="pt")

outputs = model.generate(inputs['input_ids'], max_length=512, num_return_sequences=1)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Output:", decoded_output)

In [ ]:
import shutil
shutil.make_archive('fine_tune_Llama_2', 'zip', 'fine_tune_Llama_2')
